#Приоритет 2030, Сеченовский университет
##"Классификатор бактериального и вирусного воспаления по анализу гемограмм с использованием методов машинного обучения"
*Венерин А.А., Каневский Н.И.*
###Институт цифровой медицины, ИКМ им. Н.В. Склифосовского



### Аннотация

В работе представлен вариант упрощенного гематологического классификатора, группирующий объекты по группам нормы, бактериального и вирусного воспаления. В работе представлены реализации методов машинного обучения с учителем: решающее дерево и случайный лес. Полученные результаты могут лечь в основу написания полноценного классификатора-помощника врача, существенно ускоряющего процесс анализа гемограмм пациентов.



### Реализаиця решения

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from tqdm import tqdm

In [2]:
data = pd.read_csv('blood_sample.csv')

In [3]:
data = data.drop(['Unnamed: 0'], axis=1) # удаляем ненужный столбец
data.head(15)

,hb,rbc,wbc,seg_anc,ban_anc,seg_anc_%,ban_anc_%,anc,"ly, %",alc,"eo, %",aec,"ba, %",abc,"mo, %",amc,plt,esr,norm
0,130.61,5.06,4.43,2.34,0.05,48.1,1.1,2.39,32.3,1.57,0.0,0.00,0.4,0.02,9.4,0.45,328.41,13.0,1.0
1,148.76,4.68,14.11,8.34,0.05,69.9,0.4,8.40,44.9,3.35,4.2,0.50,0.0,0.00,15.6,1.86,431.13,2.0,3.0
2,147.70,3.53,15.94,11.03,0.65,69.4,4.1,11.68,16.1,2.56,3.3,0.52,0.1,0.02,7.3,1.16,367.31,21.0,2.0
3,148.88,3.57,5.89,3.44,0.25,68.4,5.0,3.69,33.5,1.68,0.1,0.00,0.4,0.02,9.9,0.50,213.71,8.0,1.0
4,155.48,3.62,14.66,10.32,0.80,64.0,5.0,11.13,15.8,2.55,1.9,0.31,1.0,0.16,3.2,0.51,384.23,23.0,2.0
5,137.22,3.62,7.35,5.02,0.24,68.0,3.2,5.26,20.2,1.49,3.9,0.29,0.6,0.04,3.6,0.27,388.65,12.0,1.0
6,138.42,5.01,13.27,8.80,0.53,58.0,3.5,9.33,14.8,2.24,4.2,0.63,0.9,0.14,6.1,0.93,381.98,19.0,2.0
7,143.78,3.91,15.07,8.90,0.19,71.7,1.5,9.09,47.4,3.89,0.7,0.08,0.8,0.10,15.4,1.91,296.68,14.0,3.0
8,148.39,4.12,13.89,9.82,0.77,53.7,4.2,10.60,11.6,2.13,0.7,0.13,0.4,0.07,5.3,0.96,406.04,15.0,2.0
9,132.74,4.75,10.13,5.67,0.28,60.2,3.0,5.95,25.7,2.42,3.7,0.35,0.1,0.01,14.9,1.40,365.19,11.0,3.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   hb         3000 non-null   float64
 1   rbc        3000 non-null   float64
 2   wbc        3000 non-null   float64
 3   seg_anc    3000 non-null   float64
 4   ban_anc    3000 non-null   float64
 5   seg_anc_%  3000 non-null   float64
 6   ban_anc_%  3000 non-null   float64
 7   anc        3000 non-null   float64
 8   ly, %      3000 non-null   float64
 9   alc        3000 non-null   float64
 10  eo, %      3000 non-null   float64
 11  aec        3000 non-null   float64
 12  ba, %      3000 non-null   float64
 13  abc        3000 non-null   float64
 14  mo, %      3000 non-null   float64
 15  amc        3000 non-null   float64
 16  plt        3000 non-null   float64
 17  esr        3000 non-null   float64
 18  norm       3000 non-null   float64
dtypes: float64(19)
memory usage: 445.4 KB


*   hb - гемоглобин, г/л
*   rbc - эритроциты, 10^12/л
*   wbc - лейкоциты, 10^9/л
*   seg_anc - сегментоядерные нейтрофилы, 10^9/л
*   ban_anc - палочкоядерные нейтрофилы, 10^9/л
*   seg_anc_% - сегментоядерные нейтрофилы, %
*   ban_anc_% - палочкоядерные нейтрофилы, %
*   anc - нейтрофилы, 10^9/л
*   ly, % - лимфоциты, %
*   alc - лимфоциты, 10^9/л
*   eo, % - эозинофилы, %
*   aec - эозинофилы, 10^9/л
*   ba, % - базофилы, %
*   abc - базофилы, 10^9/л
*   mo, % - моноциты, %
*   amc - моноциты, 10^9/л
*   plt - тромбоциты, 10^9/л
*   esr - СОЭ, мм/час
*   norm - норма: 1, бактериальное воспаление: 2, вирусное воспаление: 3

В данных нет пропусков, типы данных не требуют изменений.

Готовим выборки для обучения.

In [5]:
target = data.norm
features = data.drop(['norm'], axis=1)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state=1)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size = 0.25, random_state=1)

### Проверим выборки на дисбаланс

In [6]:
print(f'features_train: {features_train.shape}')
print(f'features_valid: {features_valid.shape}')
print(f'features_test: {features_test.shape}')
print(data['norm'].value_counts())

features_train: (1800, 18)
features_valid: (600, 18)
features_test: (600, 18)
1.0    1000
3.0    1000
2.0    1000
Name: norm, dtype: int64


Размерность выборок совпадает, дисбаланса таргетов не обнаружено.

###Модели

*Поскольку при создании датасета использовалась технология OrdinalEncoder, столбец target содержит значения в формате:* 


{'норма' : 1, 'бактериальная инфекция' : 2, 'вирусная инфекция' : 3}.


*Данный тип кодирования категориальных данных оптимален для ансамблевых моделей машинного обучения, но категорически не подходит для реализации логистической регрессии.*


*В связи с данной особенностью данных было принято решение в первую очередь проанализировать классификаторы "решающее дерево" и "случайный лес", а к логистической регрессии перейти только в случае неудачи с реализацией выбранных моделей.* 

#### Решающее дерево

Реализуем подбор гиперпараметров для решающего дерева: максимальная глубина.

In [7]:
best_f1 = 0
for depth in tqdm(range(1,21,1)):
    model = DecisionTreeClassifier(random_state=1, max_depth=depth, criterion='gini')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    if f1_score(target_valid, predicted_valid, average='macro') > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid,average='macro')
        best_depth = depth
        print("F1:", round(f1_score(target_valid, predicted_valid, average='macro'), 4), "depth:", best_depth)   

 30%|███       | 6/20 [00:00<00:00, 57.51it/s]

F1: 0.5302 depth: 1
F1: 0.962 depth: 2
F1: 0.9741 depth: 3
F1: 0.9758 depth: 4
F1: 0.9776 depth: 6


100%|██████████| 20/20 [00:00<00:00, 58.95it/s]


In [8]:
f1_score(target_test, model.predict(features_test), average = 'macro')

0.9725172672275946

Решающее дерево показывает отличные результаты по f1-мере: 0.974. Всё-таки прибегнем к более сложной модели - случайному лесу.

#### Случайный лес

Реализуем подбор гиперпараметров для случайного леса: максимальная глубина, количество деревьев. Обучаем модель на тренировочной выборке и оцениваем f1 меру на валидационных данных.

В f1_score устанавливаем параметр average='macro', поскольку классификация мультиклассовая.

In [9]:
best_f1 = 0
for depth in tqdm(range(1,11,1)):
  for estimators in range(1,21,1):
    model = RandomForestClassifier(random_state=1, max_depth=depth, n_estimators=estimators)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    if f1_score(target_valid, predicted_valid, average = 'macro') > best_f1:
      best_f1 = f1_score(target_valid, predicted_valid, average='macro')
      best_depth = depth
      best_estimators = estimators
      print("F1:", round(f1_score(target_valid, predicted_valid, average='macro'), 4), "depth:", best_depth, 
            "estimators:", best_estimators)   

  0%|          | 0/10 [00:00<?, ?it/s]

F1: 0.5345 depth: 1 estimators: 1
F1: 0.793 depth: 1 estimators: 2
F1: 0.9417 depth: 1 estimators: 3
F1: 0.971 depth: 1 estimators: 4


 10%|█         | 1/10 [00:00<00:04,  1.89it/s]

F1: 0.9725 depth: 2 estimators: 3
F1: 0.9742 depth: 2 estimators: 4
F1: 0.9743 depth: 2 estimators: 5


 20%|██        | 2/10 [00:01<00:04,  1.61it/s]

...

 60%|██████    | 6/10 [00:04<00:02,  1.39it/s]

F1: 0.981 depth: 7 estimators: 3


 80%|████████  | 8/10 [00:08<00:02,  1.43s/it]

F1: 0.9827 depth: 9 estimators: 14


100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


Проводим окончательный мониторинг эффективности модели на тестовых данных.

In [10]:
f1_score(target_test, model.predict(features_test), average = 'macro')

0.9789753982006323

Случайный лес немного улучшил и так отличные показатели: f1_score = 0.9789

Можно сказать, что модель практически безошибочно определяет принадлежность нового объекта к классу.

####  DummyClassifier для проверки модели на адекватность

In [11]:
dummy_model = DummyClassifier(strategy='uniform', random_state=1)
dummy_model.fit(features_train, target_train)
f1_score(dummy_model.predict(features_test), target_test, average='macro')

0.3171423696473806

DummyClassifier не справляется с задачей и показывает значительно худшие результаты, сравнивая с моедлью случайного леса с подобранными гиперпараметрами.

###Вывод
Продемонстрированные возможности выбранных моделей машинного обучения (DecisionTreeClassifier и RandomForestClassifier) могут существенно ускорить внедрение подобного программного обеспечения в поликлинические учреждения здравоохранения и значительно сократить время и функцию ошибки дифференциальной диагностики воспалительных заболеваний, а также увеличить точность назначения антибактериальной терапии против именно бактериальных инфекций.

###Список литературы

1.	Adam H. Agbaria, Guy Beck Rosen, Itshak Lapidot, Daniel H. Rich, Mahmoud Huleihel, Shaul Mordechai, Ahmad Salman, and Joseph Kapelushnik, Differential Diagnosis of the Etiologies of Bacterial and Viral Infections Using Infrared Microscopy of Peripheral Human Blood Samples and Multivariate Analysis, Analytical Chemistry 2018 90 (13), 7888-7895, DOI: 10.1021/acs.analchem.8b00017
2.	Gunčar, G., Kukar, M., Notar, M. et al. An application of machine learning to haematological diagnosis. Sci Rep 8, 411 (2018). https://doi.org/10.1038/s41598-017-18564-8
3.	Largman-Chalamish M, Wasserman A, Silberman A, Levinson T, Ritter O, Berliner S, et al. (2022) Differentiating between bacterial and viral infections by estimated CRP velocity. PLoS ONE 17(12): e0277401. https://doi.org/10.1371/journal. pone.0277401
4.	Thomas J, Pociute A, Kevalas R, Malinauskas M, Jankauskaite L. Blood biomarkers differentiating viral versus bacterial pneumonia aetiology: a literature review. Ital J Pediatr. 2020 Jan 9;46(1):4. doi: 10.1186/s13052-020-0770-3. PMID: 31918745; PMCID: PMC6953310.
5.	Sweeney TE, Wong HR, Khatri P. Robust classification of bacterial and viral infections via integrated host gene expression diagnostics. Sci Transl Med. 2016 Jul 6;8(346):346ra91. doi: 10.1126/scitranslmed.aaf7165. PMID: 27384347; PMCID: PMC5348917.
